### How about portfolio selection based on ticker's past sharpe ratios (3, 6, 12 months) vs sharpe ratios of VGT? 

### Portfolio Selection
- Ask Google AI return the top 10 holdings of these index ETFs
- Ask Google AI to combine all the tickers into a unique list
- Plot the tickers vs benchmark VGT
- Keep tickers that outperform VGT in 6 & 12 months period

etf_list = [  
    # U.S. Technology Sector ETFs  
    'VGT',
    'FTEC',
    
    # U.S. Large-Cap Growth ETFs
    'QQQ',
    'VUG',
    'IWF',
    'SCHG',
    'SPYG',
    
    # U.S. Mid-Cap Growth ETFs
    'VOT',
    'IWP',
    'IJK',
    
    # Global & International ETFs
    'VT',
    'VXUS',
    'VEA',
    
    # Single-Country & Thematic Index ETFs
    'EWC',
    'PNQI'
]

In [15]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import pprint
import inspect  # <--- ADD THIS LINE
from IPython.display import display, Markdown

# --- 1. PANDAS & IPYTHON OPTIONS ---
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 3000)
pd.set_option('display.float_format', '{:.6f}'.format)
%load_ext autoreload
%autoreload 2

# --- 2. PROJECT PATH CONFIGURATION ---
NOTEBOOK_DIR = Path.cwd()
PARENT_DIR = NOTEBOOK_DIR
ROOT_DIR = NOTEBOOK_DIR.parent  # Adjust if your notebook is in a 'notebooks' subdirectory
DATA_DIR = ROOT_DIR / 'data'
SRC_DIR = ROOT_DIR / 'src'

# Add 'src' to the Python path to import custom modules
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- 3. IMPORT CUSTOM MODULES ---
import utils
import plotting_utils

# --- 4. INITIAL_CAPITAL ---
INITIAL_CAPITAL = 100000

# --- 5. RISK FREE ANNUAL RATE ---
RISK_FREE_ANNUAL_RATE = 0.04

# --- 6. VERIFICATION ---
print("--- Path Configuration ---")
print(f"✅ Project Root: {ROOT_DIR}")
print(f"✅ Parent Dir:   {PARENT_DIR}")
print(f"✅ Notebook Dir: {NOTEBOOK_DIR}")
print(f"✅ Data Dir:     {DATA_DIR}")
print(f"✅ Source Dir:   {SRC_DIR}")
assert all([ROOT_DIR.exists(), DATA_DIR.exists(), SRC_DIR.exists()]), "A key directory was not found!"

print("\n--- Module Verification ---")
print(f"✅ Successfully imported 'utils' and 'plotting_utils'.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
--- Path Configuration ---
✅ Project Root: c:\Users\ping\Files_win10\python\py311\stocks
✅ Parent Dir:   c:\Users\ping\Files_win10\python\py311\stocks\notebooks_PyPortfOpt
✅ Notebook Dir: c:\Users\ping\Files_win10\python\py311\stocks\notebooks_PyPortfOpt
✅ Data Dir:     c:\Users\ping\Files_win10\python\py311\stocks\data
✅ Source Dir:   c:\Users\ping\Files_win10\python\py311\stocks\src

--- Module Verification ---
✅ Successfully imported 'utils' and 'plotting_utils'.


In [16]:
df = pd.read_parquet(DATA_DIR / 'df_adj_close.parquet')
print(f'df:\n{df}')

df:
Ticker              A        AA       AAL      AAON       AAPL       ABBV     ABEV       ABNB        ABT      ACGL      ACHR       ACI        ACM        ACN       ACWI      ACWX       ADBE       ADC        ADI       ADM        ADP       ADSK      ADT        AEE      AEG        AEM        AEP        AER       AES        AFG        AFL      AFRM       AGCO       AGG       AGI     AGNC       AIG       AIQ      AIRR        AIT        AIZ        AJG      AKAM        AL        ALB       ALC      ALGM       ALGN       ALK        ALL       ALLE      ALLY       ALNY      ALSN        ALV        AM       AMAT      AMCR        AMD        AME       AMGN       AMH      AMLP        AMP        AMT       AMX       AMZN         AN       ANET        AON       AOS       APA        APD       APG        APH        APO        APP       APPF      APTV        AR      ARCC        ARE       ARES       ARGX      ARKK      ARMK        ARW       ASML       ASND        ASR      ASTS       ASX       ATI        AT

In [17]:
ticker_list = [
# 'MSFT', 'AAPL', 'NVDA', 'GOOG', 'AMZN',
# 'META', 'LLY', 'TSM', 'AVGO', 'NVO',
# 'V', 'TSLA', 'ASML', 'LVMUY', 'ORCL',
# 'TM', 'CRM', 'COST', 'ADBE', 'NSRGY',
# 'SHEL', 'AMD', 'NFLX', 'ACN', 'AZN',
# 'RHHBY', 'CSCO', 'TCEHY', 'ORLY', 'BKNG',
# 'MELI', 'RY', 'TD', 'SHOP', 'TDG', 'APH',
# 'SNPS', 'BN', 'IDXX', 'CDNS', 'ENB', 'BMO',
# 'AJG', 'CP', 'ANET', 'CRWD', 'ABNB', 'CNQ',
# 'BNS', 'SQ', 'DECK', 'TTD', 'ANSS', 'COIN',
# 'VRT', 'FDS', 'TRGP', 'ON', 'BLDR', 'WSM',
# 'ROKU', 'DKNG', 'CSU.TO', 'EME', 'STLD',
# 'RBLX', 'U', 'PATH', 'MSTR',
]

# ticker_list = [
# 'MSFT', 'NVDA', 'TSM', 'AVGO', 'ORCL',
# 'SHEL', 'AMD', 'NFLX', 'CSCO', 'BKNG',
# 'MELI', 'TD', 'TDG', 'APH','IDXX', 
# 'CDNS', 'BMO', 'CRWD', 'COIN','VRT', 
# 'EME', 'RBLX', 'U', 'MSTR',
# ]

ticker_list = [
'NVDA', 'AVGO', 'ORCL', 'NFLX', 'APH',
'CRWD', 'EME', 'RBLX', 'U', 'MSTR',
]

In [18]:
# plot_comparative_performance(df, ticker_list)
plotting_utils.plot_comparative_performance(df, ticker_list, benchmark='VGT')

In [19]:
file_list = utils.get_recent_files(
    directory_path = DATA_DIR,
    extension = 'parquet', 
    prefix = None,
    contains_pattern = 'df_finviz_merged_stocks_etfs',
    count = None
)

print(f'file_list:\n{file_list}')

file_list:
['2025-07-29_df_finviz_merged_stocks_etfs.parquet', '2025-07-28_df_finviz_merged_stocks_etfs.parquet', '2025-07-25_df_finviz_merged_stocks_etfs.parquet', '2025-07-24_df_finviz_merged_stocks_etfs.parquet', '2025-07-23_df_finviz_merged_stocks_etfs.parquet', '2025-07-22_df_finviz_merged_stocks_etfs.parquet', '2025-07-21_df_finviz_merged_stocks_etfs.parquet', '2025-07-18_df_finviz_merged_stocks_etfs.parquet', '2025-07-17_df_finviz_merged_stocks_etfs.parquet', '2025-07-16_df_finviz_merged_stocks_etfs.parquet', '2025-07-15_df_finviz_merged_stocks_etfs.parquet', '2025-07-14_df_finviz_merged_stocks_etfs.parquet', '2025-07-11_df_finviz_merged_stocks_etfs.parquet', '2025-07-10_df_finviz_merged_stocks_etfs.parquet', '2025-07-09_df_finviz_merged_stocks_etfs.parquet', '2025-07-08_df_finviz_merged_stocks_etfs.parquet', '2025-07-07_df_finviz_merged_stocks_etfs.parquet', '2025-07-03_df_finviz_merged_stocks_etfs.parquet', '2025-06-06_df_finviz_merged_stocks_etfs.parquet', '2025-06-05_df_finv

In [20]:
df_finviz = pd.read_parquet(DATA_DIR / file_list[0])
print(f'df_finviz:\n{df_finviz}')

df_finviz:
       No.                                          Company               Index                  Sector                        Industry Country Exchange                                               Info  MktCap AUM, M  Rank  Market Cap, M       P/E   Fwd P/E      PEG       P/S       P/B       P/C      P/FCF   Book/sh   Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %       EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %      Sales, M     Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M     Float, M   Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M     ROA %      ROE %    ROIC %   Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf 3D %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %     Beta      ATR  ATR/Price %  Volatility W % 

In [21]:

_df = df_finviz.loc[ticker_list]
print(_df)

      No.                   Company               Index                  Sector                        Industry Country Exchange                                               Info  MktCap AUM, M  Rank  Market Cap, M        P/E   Fwd P/E      PEG        P/S        P/B         P/C      P/FCF    Book/sh   Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %        EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %    EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %      Sales, M    Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M     Float, M   Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M      ROA %       ROE %     ROIC %   Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %   Oper M %   Profit M %  Perf 3D %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %     Beta       ATR  ATR/Price %  Volatility W %  Volatility M %   SMA

In [22]:
column_list = ['Company', 'Info', 'Rank', 'MktCap AUM, M', 'ATR/Price %', 'Price']
_df = _df.loc[:, column_list]
print(f"df:\n{_df}")

df:
                       Company                                               Info  Rank  MktCap AUM, M  ATR/Price %       Price
NVDA               NVIDIA Corp                         Technology, Semiconductors     1 4282440.000000     2.170816  175.510000
AVGO              Broadcom Inc                         Technology, Semiconductors     8 1398910.000000     2.565396  297.420000
ORCL               Oracle Corp              Technology, Software - Infrastructure    17  702150.000000     2.556204  249.980000
NFLX               Netflix Inc              Communication Services, Entertainment    23  496630.000000     2.484727 1168.740000
APH              Amphenol Corp                  Technology, Electronic Components   119  128580.000000     2.639825  105.310000
CRWD  Crowdstrike Holdings Inc              Technology, Software - Infrastructure   137  116030.000000     3.119160  465.510000
EME           Emcor Group, Inc            Industrials, Engineering & Construction   513   28480.0000

In [23]:
df_buy = _df.loc[:, ['Info', 'MktCap AUM, M', 'ATR/Price %', 'Price']]
print(f'df_buy:\n{df_buy}')

df_buy:
                                                   Info  MktCap AUM, M  ATR/Price %       Price
NVDA                         Technology, Semiconductors 4282440.000000     2.170816  175.510000
AVGO                         Technology, Semiconductors 1398910.000000     2.565396  297.420000
ORCL              Technology, Software - Infrastructure  702150.000000     2.556204  249.980000
NFLX              Communication Services, Entertainment  496630.000000     2.484727 1168.740000
APH                   Technology, Electronic Components  128580.000000     2.639825  105.310000
CRWD              Technology, Software - Infrastructure  116030.000000     3.119160  465.510000
EME             Industrials, Engineering & Construction   28480.000000     2.429939  636.230000
RBLX  Communication Services, Electronic Gaming & Mu...   80260.000000     3.498690  118.330000
U                    Technology, Software - Application   13530.000000     5.500922   32.540000
MSTR                 Technology,

In [24]:
# ------------------------------------------------------------------
# 1.  Market-Capitalisation weight
# ------------------------------------------------------------------
tot_mcap = df_buy['MktCap AUM, M'].sum()
df_buy['MktCap Weight'] = df_buy['MktCap AUM, M'] / tot_mcap

# ------------------------------------------------------------------
# 2.  Inverse-ATR weight (smaller ATR/Price % → larger weight)
# ------------------------------------------------------------------
inv_atr = 1 / df_buy['ATR/Price %']          # inverse
tot_inv = inv_atr.sum()
df_buy['Inv ATR/Price Weight'] = inv_atr / tot_inv

# 1. Sum the two raw weights
df_buy['Portfolio Weight'] = (
        df_buy['MktCap Weight'] + df_buy['Inv ATR/Price Weight']
)

# 2. Normalize so the summed weights equal 1.0
df_buy['Portfolio Weight'] /= df_buy['Portfolio Weight'].sum()

print(f'df_buy:\n{df_buy}')
# print(df_buy[['MktCap AUM, M', 'MkrCap Weight',
#               'ATR/Price %', 'Inv ATR/Price Weight']])

df_buy:
                                                   Info  MktCap AUM, M  ATR/Price %       Price  MktCap Weight  Inv ATR/Price Weight  Portfolio Weight
NVDA                         Technology, Semiconductors 4282440.000000     2.170816  175.510000       0.581939              0.133116          0.357527
AVGO                         Technology, Semiconductors 1398910.000000     2.565396  297.420000       0.190097              0.112641          0.151369
ORCL              Technology, Software - Infrastructure  702150.000000     2.556204  249.980000       0.095415              0.113046          0.104231
NFLX              Communication Services, Entertainment  496630.000000     2.484727 1168.740000       0.067487              0.116298          0.091893
APH                   Technology, Electronic Components  128580.000000     2.639825  105.310000       0.017473              0.109466          0.063469
CRWD              Technology, Software - Infrastructure  116030.000000     3.119160  4

In [25]:
df_buy_sorted_by_price = df_buy.sort_values('Price', ascending=False)
sum_MktCap_Weight = df_buy['MktCap Weight'].sum()
sum_Inv_ATR_Price_Weight  = df_buy['Inv ATR/Price Weight'].sum()
sum_portfolio_weights = df_buy['Portfolio Weight'].sum()
print(f'Sum MktCap Weights: {sum_MktCap_Weight:.2f}')
print(f'sum Inv ATR/Price Weights: {sum_Inv_ATR_Price_Weight:.2f}')
print(f'Sum of Portfolio Weights: {sum_portfolio_weights:.2f}')
print(f'\ndf_buy_sorted_by_price:\n{df_buy_sorted_by_price}')


Sum MktCap Weights: 1.00
sum Inv ATR/Price Weights: 1.00
Sum of Portfolio Weights: 1.00

df_buy_sorted_by_price:
                                                   Info  MktCap AUM, M  ATR/Price %       Price  MktCap Weight  Inv ATR/Price Weight  Portfolio Weight
NFLX              Communication Services, Entertainment  496630.000000     2.484727 1168.740000       0.067487              0.116298          0.091893
EME             Industrials, Engineering & Construction   28480.000000     2.429939  636.230000       0.003870              0.118921          0.061395
CRWD              Technology, Software - Infrastructure  116030.000000     3.119160  465.510000       0.015767              0.092644          0.054205
MSTR                 Technology, Software - Application  111910.000000     4.203618  394.660000       0.015207              0.068743          0.041975
AVGO                         Technology, Semiconductors 1398910.000000     2.565396  297.420000       0.190097              0.112641

#######################################  
#######################################  
#######################################  

In [ ]:
df_finviz

<class 'pandas.core.frame.DataFrame'>
Index: 1496 entries, NVDA to VTHR
Columns: 139 entries, No. to Omega 250d
dtypes: float64(120), int64(3), object(16)
memory usage: 1.6+ MB


In [52]:
# 1. Select the four Sharpe columns plus the index so we can keep the ticker names
cols = ['Sharpe 30d', 'Sharpe 60d', 'Sharpe 120d', 'Sharpe 250d']
tmp = df_finviz[cols]

# 2. Grab VGT’s row and turn it into a 1-row DataFrame for broadcasting
vgt_vals = tmp.loc['VGT'].to_frame().T   # shape (1,4)

# 3. Boolean mask: True wherever all of the 4 columns is strictly greater than VGT’s value
mask = (tmp > vgt_vals.values).all(axis=1)

# 4. Pull the rows that satisfy that mask
higher_than_vgt = df_finviz[mask]

# 5. See how many there are and inspect
print(higher_than_vgt[cols])

# 6. see VGT
print(vgt_vals)

      Sharpe 30d  Sharpe 60d  Sharpe 120d  Sharpe 250d
NVDA    6.357041    6.075524     1.530764     1.162276
GS      6.546094    5.041323     0.845836     1.206676
CAT     8.060878    5.845338     1.285297     0.835323
C       7.237934    5.423134     1.096960     1.223508
RCL     6.675466    5.034419     1.142291     1.876783
EMR     6.750432    6.205214     1.126928     0.862911
BK      6.404267    5.797693     1.420655     1.930862
EME     7.236536    5.742171     1.781008     1.508421
SOFI    8.228402    5.585931     1.431250     2.049570
WDC     7.994936    7.511837     1.651253     0.999799
CLS     6.506697    6.102663     1.272912     2.241766
IYW     6.454683    5.143800     0.773091     0.870751
TPR     7.667868    5.091334     1.917275     2.555795
TRMB    8.916405    5.915808     0.784041     1.366743
MINT    7.330788    5.020055     1.357147     2.041952
CX      7.202490    5.533705     2.035816     0.942427
LITE    6.370615    5.884106     0.908765     1.470813
CAR     6.

In [53]:
# Columns you want to keep in the final table
extra_cols = [
    'Company',
    # 'Index',
    # 'Sector',
    'Industry',
    'Info',
    'MktCap AUM, M',
    'Beta',
    'ATR/Price %',
    'Price'
]

# Combine the Sharpe columns you already selected with the extra ones
final_cols = extra_cols + ['Sharpe 30d', 'Sharpe 60d', 'Sharpe 120d', 'Sharpe 250d']

# Slice the rows you already filtered and add the new columns
higher_than_vgt = higher_than_vgt[final_cols]

# Done – higher_than_vgt now contains only the rows with higher Sharpe
# values than VGT and the requested extra fields

In [54]:
print(higher_than_vgt)

                                       Company                             Industry                                               Info  MktCap AUM, M     Beta  ATR/Price %      Price  Sharpe 30d  Sharpe 60d  Sharpe 120d  Sharpe 250d
NVDA                               NVIDIA Corp                       Semiconductors                         Technology, Semiconductors 4282440.000000 2.140000     2.170816 175.510000    6.357041    6.075524     1.530764     1.162276
GS                    Goldman Sachs Group, Inc                      Capital Markets                         Financial, Capital Markets  224600.000000 1.370000     1.801962 731.980000    6.546094    5.041323     0.845836     1.206676
CAT                            Caterpillar Inc  Farm & Heavy Construction Machinery   Industrials, Farm & Heavy Construction Machinery  202260.000000 1.400000     1.737007 430.050000    8.060878    5.845338     1.285297     0.835323
C                                Citigroup Inc                  Bank

In [55]:
# remove_tickers should be a list/tuple/Series of tickers you want to drop
remove_tickers = ['MINT', 'BK']

# 1-line drop
higher_than_vgt = higher_than_vgt.drop(index=remove_tickers)

# If you want to be safe in case a ticker isn't present:
# higher_than_vgt = higher_than_vgt.drop(index=remove_tickers, errors='ignore')

In [56]:
# ------------------------------------------------------------------
# 1.  Market-Capitalisation weight
# ------------------------------------------------------------------
tot_mcap = higher_than_vgt['MktCap AUM, M'].sum()
higher_than_vgt['MktCap Weight'] = higher_than_vgt['MktCap AUM, M'] / tot_mcap

# ------------------------------------------------------------------
# 2.  Inverse-ATR weight (smaller ATR/Price % → larger weight)
# ------------------------------------------------------------------
inv_atr = 1 / higher_than_vgt['ATR/Price %']          # inverse
tot_inv = inv_atr.sum()
higher_than_vgt['Inv ATR/Price Weight'] = inv_atr / tot_inv

# 1. Sum the two raw weights
higher_than_vgt['Portfolio Weight'] = (
        higher_than_vgt['MktCap Weight'] + higher_than_vgt['Inv ATR/Price Weight']
)

# 2. Normalize so the summed weights equal 1.0
higher_than_vgt['Portfolio Weight'] /= higher_than_vgt['Portfolio Weight'].sum()

print(f'higher_than_vgt:\n{higher_than_vgt}')
# print(higher_than_vgt[['MktCap AUM, M', 'MkrCap Weight',
#               'ATR/Price %', 'Inv ATR/Price Weight']])

higher_than_vgt:
                          Company                             Industry                                               Info  MktCap AUM, M     Beta  ATR/Price %      Price  Sharpe 30d  Sharpe 60d  Sharpe 120d  Sharpe 250d  MktCap Weight  Inv ATR/Price Weight  Portfolio Weight
NVDA                  NVIDIA Corp                       Semiconductors                         Technology, Semiconductors 4282440.000000 2.140000     2.170816 175.510000    6.357041    6.075524     1.530764     1.162276       0.815128              0.065553          0.440341
GS       Goldman Sachs Group, Inc                      Capital Markets                         Financial, Capital Markets  224600.000000 1.370000     1.801962 731.980000    6.546094    5.041323     0.845836     1.206676       0.042751              0.078972          0.060861
CAT               Caterpillar Inc  Farm & Heavy Construction Machinery   Industrials, Farm & Heavy Construction Machinery  202260.000000 1.400000     1.737007

In [57]:
# Ensure the column is numeric (it may be a string if it contains commas or '%')
higher_than_vgt['Portfolio Weight'] = (
    higher_than_vgt['Portfolio Weight']
    .astype(str)
    .str.replace(',', '', regex=False)
    .str.rstrip('%')
    .astype(float)
)

# Sort descending by Portfolio Weight
higher_than_vgt = higher_than_vgt.sort_values('Portfolio Weight', ascending=False)

print(f'higher_than_vgt:\n{higher_than_vgt}')

higher_than_vgt:
                          Company                             Industry                                               Info  MktCap AUM, M     Beta  ATR/Price %      Price  Sharpe 30d  Sharpe 60d  Sharpe 120d  Sharpe 250d  MktCap Weight  Inv ATR/Price Weight  Portfolio Weight
NVDA                  NVIDIA Corp                       Semiconductors                         Technology, Semiconductors 4282440.000000 2.140000     2.170816 175.510000    6.357041    6.075524     1.530764     1.162276       0.815128              0.065553          0.440341
IYW   iShares U.S. Technology ETF                 Exchange Traded Fund  Financial, Exchange Traded Fund, US Equities -...   22660.000000 1.240000     1.143394 181.040000    6.454683    5.143800     0.773091     0.870751       0.004313              0.124458          0.064385
GS       Goldman Sachs Group, Inc                      Capital Markets                         Financial, Capital Markets  224600.000000 1.370000     1.801962

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 644 entries, 2023-01-03 to 2025-07-29
Columns: 1524 entries, A to ZWS
dtypes: float64(1524)
memory usage: 7.5 MB


### Major error, you are looking at the past, of cause the data will be good

In [64]:
import pandas as pd

# 1. 250 trading-days ago (≈ one business-year)
start_date = df.index[-250]

# 2. Portfolio definition
weights = higher_than_vgt['Portfolio Weight'] / 100.0          # convert % → fraction
tickers = weights.index                                        # these are the tickers we hold

# 3. Slice price matrix for the 250-day window
prices = df.loc[start_date:, tickers].copy()                   # shape (251, n_tickers)

# 4. Normalise prices to “1.0 on day-0”
norm = prices.div(prices.iloc[0])                            # each column starts at 1

# 5. Apply the portfolio weights
weighted = norm.mul(weights)                                 # each day: w_i * (P_i / P_i0)

# 6. Portfolio level
portfolio_value = weighted.sum(axis=1) * 1000                # scale to 1000 initial capital
portfolio_value.name = 'Portfolio_Value'

# 7. Result: daily time-series
print(portfolio_value)

Date
2024-07-30   10.000000
2024-07-31   10.672918
2024-08-01   10.120084
2024-08-02    9.772164
2024-08-05    9.369904
                ...   
2025-07-23   16.666394
2025-07-24   16.786970
2025-07-25   16.909166
2025-07-28   17.068026
2025-07-29   17.102596
Name: Portfolio_Value, Length: 250, dtype: float64


In [65]:
# 1. VGT price series for the same 250-day window
vgt_prices = df.loc[start_date:, 'VGT']

# 2. Normalise to 1000 initial capital
vgt_value = (vgt_prices / vgt_prices.iloc[0]) * 1000
vgt_value.name = 'VGT_Value'

# 3. Display / compare
print(vgt_value)

Date
2024-07-30   1000.000000
2024-07-31   1034.986346
2024-08-01    997.576981
2024-08-02    973.106867
2024-08-05    939.050902
                 ...    
2025-07-23   1254.896861
2025-07-24   1258.449890
2025-07-25   1261.306965
2025-07-28   1271.288413
2025-07-29   1269.072348
Name: VGT_Value, Length: 250, dtype: float64
